In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [28]:
df = pd.read_csv('/content/drive/MyDrive/CSCE464/datasets/Iris.csv')
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


#Apply one-hot encoding to species column

In [29]:
#0 for setosa, 1 for versicolor, 2 for virginica
df['Iris-setosa'] = df['Species'].apply(lambda x:1 if x == 'Iris-setosa' else 0)
df['Iris-versicolor'] = df['Species'].apply(lambda x:2 if x == 'Iris-versicolor' else 0 )
df['Iris-virginica'] = df['Species'].apply(lambda x:3 if x == 'Iris-virginica' else 0)

df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,5.1,3.5,1.4,0.2,Iris-setosa,1,0,0
1,2,4.9,3.0,1.4,0.2,Iris-setosa,1,0,0
2,3,4.7,3.2,1.3,0.2,Iris-setosa,1,0,0
3,4,4.6,3.1,1.5,0.2,Iris-setosa,1,0,0
4,5,5.0,3.6,1.4,0.2,Iris-setosa,1,0,0
...,...,...,...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica,0,0,3
146,147,6.3,2.5,5.0,1.9,Iris-virginica,0,0,3
147,148,6.5,3.0,5.2,2.0,Iris-virginica,0,0,3
148,149,6.2,3.4,5.4,2.3,Iris-virginica,0,0,3


In [30]:
df = df.drop(['Id', 'Species'], axis=1)

df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Iris-setosa,Iris-versicolor,Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0
...,...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,0,0,3
146,6.3,2.5,5.0,1.9,0,0,3
147,6.5,3.0,5.2,2.0,0,0,3
148,6.2,3.4,5.4,2.3,0,0,3


In [31]:
x_data = df[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]
y_data = df[['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']]

In [32]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)

In [33]:
x_train

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
67,5.8,2.7,4.1,1.0
65,6.7,3.1,4.4,1.4
32,5.2,4.1,1.5,0.1
100,6.3,3.3,6.0,2.5
120,6.9,3.2,5.7,2.3
...,...,...,...,...
102,7.1,3.0,5.9,2.1
58,6.6,2.9,4.6,1.3
97,6.2,2.9,4.3,1.3
76,6.8,2.8,4.8,1.4


In [34]:
y_train

,Iris-setosa,Iris-versicolor,Iris-virginica
67,0,2,0
65,0,2,0
32,1,0,0
100,0,0,3
120,0,0,3
...,...,...,...
102,0,0,3
58,0,2,0
97,0,2,0
76,0,2,0


In [35]:
x_test

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm
18,5.7,3.8,1.7,0.3
89,5.5,2.5,4.0,1.3
91,6.1,3.0,4.6,1.4
127,6.1,3.0,4.9,1.8
121,5.6,2.8,4.9,2.0
129,7.2,3.0,5.8,1.6
74,6.4,2.9,4.3,1.3
55,5.7,2.8,4.5,1.3
10,5.4,3.7,1.5,0.2
59,5.2,2.7,3.9,1.4


In [36]:
y_test

,Iris-setosa,Iris-versicolor,Iris-virginica
18,1,0,0
89,0,2,0
91,0,2,0
127,0,0,3
121,0,0,3
129,0,0,3
74,0,2,0
55,0,2,0
10,1,0,0
59,0,2,0


In [37]:
x_train.reset_index(drop=True)
x_test.reset_index(drop=True)
y_train.reset_index(drop=True)
y_test.reset_index(drop=True)

,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,0,0
1,0,2,0
2,0,2,0
3,0,0,3
4,0,0,3
5,0,0,3
6,0,2,0
7,0,2,0
8,1,0,0
9,0,2,0


In [38]:
#Normalizing data

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

#Softmax model in TensorFlow

In [41]:
#Define X, W, and bias

x = tf.placeholder(tf.float32, [None, 4])
y = tf.placeholder(tf.float32, [None, 3])
W = tf.Variable(tf.random_normal([4, 3]), name='weight')
b = tf.Variable(tf.random_normal([3]), name='bias')

#Define a hypothesis
hypothesis = tf.nn.softmax(tf.matmul(x, W) + b)

#Define cost function
cost  = tf.reduce_mean(-tf.reduce_sum(y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate= 0.1).minimize(cost)

#argmax returns highest index num, finds which column has number 1
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#model training
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(1001):
  sess.run(optimizer, feed_dict={x: x_train, y: y_train})

  if epoch % 100 == 0:
    _, acc = sess.run([cost, accuracy], feed_dict={x: x_train, y: y_train})
    print("Epoch: {:5}\t Cost: {:.5f}\t ACC: {:.2f}".format(epoch, _, acc))

Epoch:     0	 Cost: 2.72739	 ACC: 0.57
Epoch:   100	 Cost: 0.48246	 ACC: 0.95
Epoch:   200	 Cost: 0.36645	 ACC: 0.98
Epoch:   300	 Cost: 0.30780	 ACC: 0.98
Epoch:   400	 Cost: 0.27128	 ACC: 0.98
Epoch:   500	 Cost: 0.24599	 ACC: 0.98
Epoch:   600	 Cost: 0.22726	 ACC: 0.98
Epoch:   700	 Cost: 0.21274	 ACC: 0.98
Epoch:   800	 Cost: 0.20108	 ACC: 0.97
Epoch:   900	 Cost: 0.19148	 ACC: 0.97
Epoch:  1000	 Cost: 0.18341	 ACC: 0.97


#PyTorch version

In [45]:
x_train, x_test = torch.FloatTensor(x_train), torch.FloatTensor(x_test)
y_train, y_test = torch.FloatTensor(y_train.to_numpy()), torch.FloatTensor(y_test.to_numpy())

In [47]:
class SoftmaxClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(4, 3)
    self.softmax = nn.Softmax()

  def forward(self, x):
    return self.softmax(self.linear(x))

In [48]:
model = SoftmaxClassifier()

In [49]:
optimizer = optim.SGD(model.parameters(), lr=0.1)

print("=====Model Training=====")

for epoch in range(1001):
  x = x_train
  y = y_train

  hypothesis = model(x)
  cost = F.cross_entropy(hypothesis, y)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    prediction = hypothesis
    correct_prediction = torch.argmax(prediction, 1) == torch.argmax(y_train, 1)
    accuracy = correct_prediction.float().mean()

    print("Epoch: {:4d} Cost: {:.5f} Training ACC: {:.2f}%".format(epoch, 1000, cost.item(), accuracy*100))

=====Model Training=====
Epoch:    0 Cost: 1000.00000 Training ACC: 2.29%
Epoch:  100 Cost: 1000.00000 Training ACC: 1.54%


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch:  200 Cost: 1000.00000 Training ACC: 1.47%
Epoch:  300 Cost: 1000.00000 Training ACC: 1.42%
Epoch:  400 Cost: 1000.00000 Training ACC: 1.38%
Epoch:  500 Cost: 1000.00000 Training ACC: 1.35%
Epoch:  600 Cost: 1000.00000 Training ACC: 1.33%
Epoch:  700 Cost: 1000.00000 Training ACC: 1.31%
Epoch:  800 Cost: 1000.00000 Training ACC: 1.30%
Epoch:  900 Cost: 1000.00000 Training ACC: 1.29%
Epoch: 1000 Cost: 1000.00000 Training ACC: 1.28%
